Flight Delay Distribution Prediction
==============

**Author:** *Nicolas Haase*

# 0 Environment Insights

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
    print('Not using a high-RAM runtime')
else:
    print('You are using a high-RAM runtime!')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)

In [ ]:
import os
# Run a shell command to get CPU information
cpu_info = !cat /proc/cpuinfo
# Print the CPU information
print('\n'.join(cpu_info))

# 1 Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files

In [ ]:
! pip install properscoring

In [ ]:
! pip install shap

In [ ]:
! pip install --upgrade tensorflow

In [ ]:
! pip install scikeras

## 1.1 Used Libraries

In [ ]:
import folium
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.ticker import MultipleLocator
import numpy as np
import pandas as pd
import properscoring as ps
import seaborn as sns
import shap
import sklearn
import statsmodels.api as sm
import tensorflow as tf
import tensorflow_probability as tfp
from imblearn.under_sampling import RandomUnderSampler
from tensorflow import keras
from tensorflow.keras import layers
from scipy.stats import norm
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer
from sklearn.exceptions import ConvergenceWarning
from sklearn.neural_network import MLPRegressor
import warnings

warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

## 1.2 Data Import

In [ ]:
# Load data into a DataFrame
df = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/data/cleaned/atl_cleaned.csv")

## 1.3 Workspace Settings

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## 1.4 Feature Preparation

In [ ]:
# Rename the column "DayofMonth" to "DayOfMonth"
df.rename(columns={"DayofMonth": "DayOfMonth"}, inplace=True)

# 2 Exploratory Data Analysis

In [ ]:
df.info()

In [ ]:
# Print the first ten rows to get an overview
df.head(10)

In [ ]:
print(df.dtypes)

In [ ]:
# Print the columns
print(list(df))

In [ ]:
df.describe()

In [ ]:
print(df.shape)

In [ ]:
print(df.isnull().sum())

In [ ]:
# Specify the columns to exclude from the search
exclude_columns = ['CarrierDelay', 'WeatherDelay', 'NASDelay', 'SecurityDelay', 'LateAircraftDelay']

# Exclude specified columns from the count of missing values
missing_values_excluded = df.drop(columns=exclude_columns).isnull().sum()

# Find the column with the max number of missing values
max_missing_column = missing_values_excluded.idxmax()
max_missing_value = missing_values_excluded[max_missing_column]

print(f"The column with the minimum missing values (excluding specified columns) is '{max_missing_column}' with {max_missing_value} missing values.")

In [ ]:
# Get all unique values from the "Origin" column
num_unique_values = df['Origin'].nunique()
unique_values = sorted(df['Origin'].unique())

# Print the unique values
print("Number of unique values in the 'Origin' column:", num_unique_values)
print("Unique values in the 'Origin' column:")
for value in unique_values:
    print(value, end=", ")

In [ ]:
# Convert the 'FlightDate' column to a datetime format if it's not already
df['FlightDate'] = pd.to_datetime(df['FlightDate'])

# Group the data by month and count the data points in each group
monthly_counts = df.groupby(df['FlightDate'].dt.to_period('M')).size().reset_index(name='Count')

# Convert 'FlightDate' to a numeric representation
monthly_counts['NumericDate'] = monthly_counts['FlightDate'].dt.year + monthly_counts['FlightDate'].dt.month / 12

print(monthly_counts)

In [ ]:
# Plot the monthly counts
plt.figure(figsize=(10, 6), dpi=300)
plt.plot(monthly_counts['NumericDate'], monthly_counts['Count'], marker='o')
# plt.title('Monthly Counts of Flights')
plt.xlabel('Month')
plt.ylabel('Flights')

# Set the ticks and labels with a gap of 3 months
tick_positions = monthly_counts['NumericDate'][::2]
tick_labels = monthly_counts['FlightDate'][::2].dt.strftime('%b %Y')

plt.xticks(tick_positions, tick_labels, rotation=45)  # Rotate x-axis labels for better readability
plt.grid(True)
plt.show()

In [ ]:
# Find the maximum and minimum values in the 'Count' column
max_value = monthly_counts['Count'].max()
min_value = monthly_counts['Count'].min()

# Find the corresponding months for the maximum and minimum values
month_with_max = monthly_counts.loc[monthly_counts['Count'] == max_value, 'FlightDate'].values[0]
month_with_min = monthly_counts.loc[monthly_counts['Count'] == min_value, 'FlightDate'].values[0]

# Print the results
print("Maximum Count:", max_value)
print("Month with Maximum Count:", month_with_max)
print("Minimum Count:", min_value)
print("Month with Minimum Count:", month_with_min)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4.5), dpi=300)

arr = df['ArrDelay']
dep = df['DepDelay']

delay_range = np.arange(-50, 151, 1)

# Plot histogram for ArrDelay
ax.hist(arr, bins=delay_range, alpha=0.7, label='Arrivals', color='lightblue', edgecolor='white', align='mid', linewidth=0.5)

# Plot histogram for DepDelay
lightorange = '#ffcc66'
ax.hist(dep, bins=delay_range, alpha=0.7, label='Departures', color=lightorange, edgecolor='white', align='mid', linewidth=0.5)

# Adding labels and title
ax.set_xlabel('Delay [min]')
ax.set_ylabel('Frequency')

ax.axvline(x=0, color='grey', linestyle='--', label='Scheduled time')
ax.axvline(x=15, color='lightcoral', linestyle='--', label='15 min delay')

ax.set_xlim(-50, 150)

# Adding legend
ax.legend()

# Show the plot
plt.show()

In [ ]:
count_above_180 = (df['ArrDelay'] >= 180).sum()
print(f"Number of values in 'ArrDelay' greater than or equal to 180: {count_above_180}")

In [ ]:
above_zero_count = (df['ArrDelay'] > 0).sum()
below_zero_count = (df['ArrDelay'] < 0).sum()

if above_zero_count > below_zero_count:
    print("More values of ArrDelay are above 0.")
elif above_zero_count < below_zero_count:
    print("More values of ArrDelay are below 0.")
else:
    print("The number of values above and below 0 are equal.")

In [ ]:
count_above_180 = (df['DepDelay'] >= 180).sum()
print(f"Number of values in 'DepDelay' greater than or equal to 180: {count_above_180}")

In [ ]:
count_above_180 = ((df['DepDelay'] >= 180) | (df['ArrDelay'] >= 180)).sum()
print(f"Number of values where 'DepDelay' or 'ArrDelay' is greater than or equal to 180: {count_above_180}")

In [ ]:
above_zero_count_dep = (df['DepDelay'] > 0).sum()
below_zero_count_dep = (df['DepDelay'] < 0).sum()

if above_zero_count_dep > below_zero_count_dep:
    print("More values of DepDelay are above 0.")
elif above_zero_count_dep < below_zero_count_dep:
    print("More values of DepDelay are below 0.")
else:
    print("The number of values above and below 0 are equal for DepDelay.")

In [ ]:
mean_arr_delay = df['ArrDelay'].median()

print(f"The mean arrival delay is: {mean_arr_delay:.2f} minutes")

### Flight Network Map

In [ ]:
# Calculate value counts for each Dest
airport_counts = df['Dest'].value_counts()

# Calculate cumulative percentage
cumulative_percentage = airport_counts.cumsum() / airport_counts.sum()

# Find the top Dest contributing to 75% of the total value counts
top_airports = cumulative_percentage[cumulative_percentage <= 0.75].index

# Select the top 25 airports
top_25_airports = airport_counts.loc[top_airports].nlargest(25)

# Get all airports not in the top 25
other_airports = airport_counts[~airport_counts.index.isin(top_25_airports.index)]

# Display the result
print("Top 25 Dest contributing to 75% of total value counts:")
print(top_25_airports)

print("\nOther airports (not in the top 25):")
print(other_airports)

In [ ]:
# Select the specified columns
selected_columns = ['Origin', 'Dest', 'OriginLatitude', 'OriginLongitude', 'DestLatitude', 'DestLongitude']
selected_airports = ['ATL', 'MCO', 'LGA', 'FLL', 'MIA', 'DCA', 'TPA', 'EWR', 'ORD', 'DFW', 'DEN', 'PHL', 'BWI', 'LAX', 'CLT', 'DTW', 'BOS', 'IAH', 'LAS', 'MSY', 'JAX', 'RDU', 'BNA', 'PBI', 'AUS']

# Create a new DataFrame with only the selected columns
coordinates_df = df[selected_columns]
unique_rows_df = coordinates_df.drop_duplicates()

# Filter DataFrame to include only rows related to the top 25 airports
top_25_airports_data = unique_rows_df[unique_rows_df['Dest'].isin(selected_airports) & unique_rows_df['Origin'].isin(selected_airports)]

In [ ]:
top_25_airports_data.head()

In [ ]:
# Filter DataFrame to get data related to the origin airport (ATL)
atl_data = top_25_airports_data[top_25_airports_data['Origin'] == 'ATL']

# Calculate the geographic center of the origin airport (ATL)
center_lat = atl_data[['OriginLatitude']].mean().mean()
center_lon = atl_data[['OriginLongitude']].mean().mean()

# Create a Folium map centered at the geographic center
my_map = folium.Map(location=[center_lat, center_lon], zoom_start=4)

# Function to create a DivIcon with a white rectangle background
def create_div_icon(label):
    return folium.DivIcon(html=f"<div style='font-size: 12pt; font-weight: bold; background-color: white; padding: 2px; display: inline-block;'>{label}</div>")

# Function to add airport labels and connections
def add_airport_labels(row, color, has_labels):
    if has_labels:
        # Adding markers with white rectangle background for Origin
        folium.Marker(
            location=[row['OriginLatitude'], row['OriginLongitude']],
            icon=create_div_icon(row['Origin']),
            popup=f"Origin: {row['Origin']}"
        ).add_to(my_map)

        # Adding markers with white rectangle background for Destination
        folium.Marker(
            location=[row['DestLatitude'], row['DestLongitude']],
            icon=create_div_icon(row['Dest']),
            popup=f"Dest: {row['Dest']}"
        ).add_to(my_map)

    # Adding blue PolyLine
    folium.PolyLine([(row['OriginLatitude'], row['OriginLongitude']), (row['DestLatitude'], row['DestLongitude'])], color=color).add_to(my_map)

# Apply the function to each row in the DataFrame
unique_rows_df.apply(add_airport_labels, color="lightgreen", has_labels=False, axis=1)
top_25_airports_data.apply(add_airport_labels, color="green", has_labels=True, axis=1)

# Save the map to an HTML file
my_map.save("airport_connections_map_labels.html")

### Test Flights

In [ ]:
indices = [[956708, 139955, 512127, 1186702],
           [1252428, 1041594, 601064, 1142614],
           [436420, 665839, 139612, 425634],
           [782558, 123849, 82184, 367556],
           [681257, 644526, 448386, 393871],
           [792921, 950747, 1236213, 918685]]

columns = ['FlightDate', 'CRSDepTime', 'CRSArrTime', 'Origin', 'Dest', 'Reporting_Airline']

# Loop through each list of indices
for idx_list in indices:
    print(f"\nIndices: {idx_list}")
    
    # Select rows based on indices and append to the result DataFrame
    result_df = df.loc[idx_list, columns]

    # Print the rows for the current iteration using head()
    print(result_df)

### Features

We see no missing values in our dataset besides the features CarrierDelay, WeatherDelay, NASDelay, SecurityDelay, LateAircraftDelay. However, those five categories only hold values if the flight is delayed.

**Feature overview and meaning:**
<table>
<tr>
<th>Feature name</th>
<th>Definition</th>
<th>Feature type</th>
</tr>
<tr>
<td>Year</td>
<td>Year</td>
<td>Int</td>
<tr>
</tr>
<tr>
<td>Month</td>
<td>Month</td>
<td>Object/String</td>
<tr>
</tr>
<tr>
<td>DayOfMonth</td>
<td>Day of the Month</td>
<td>Object/String</td>
<tr>
<tr>
<td>DayOfWeek</td>
<td>Day of the Week</td>
<td>Object/String</td>
<tr>
<tr>
<td>FlightDate</td>
<td>Flight Date (yyyymmdd)</td>
<td>Int</td>
<tr>
<tr>
<td>Reporting_Airline</td>
<td>Unique Carrier Code. When the same code has been used by multiple carriers, a numeric suffix is used for earlier users, for example, PA, PA(1), PA(2)</td>
<td>Object/String</td>
<tr>
<tr>
<td>Tail_Number</td>
<td>Tail Number of the airplane</td>
<td>Object/String</td>
<tr>
<tr>
<td>Flight_Number_Reporting_Airline</td>
<td>Marital Status of User</td>
<td>Int</td>
<tr>
<tr>
<td>OriginAirportID</td>
<td>An identification number assigned by US DOT to identify a unique airport</td>
<td>Int</td>
<tr>
<tr>
<td>Origin</td>
<td>Origin Aiport</td>
<td>Float</td>
<tr>
<tr>
<td>DestAirportID</td>
<td>An identification number assigned by US DOT to identify a unique airport</td>
<td>Float</td>
<tr>
<tr>
<td>Dest</td>
<td>DestAirport</td>
<td></td>
<tr>
<tr>
<td>CRSDepTime</td>
<td>Scheduled Departure Time (local time: hhmm)</td>
<td></td>
<tr>
<tr>
<td>DepTime</td>
<td>Actual Departure Time (local time: hhmm)</td>
<td></td>
<tr>
<tr>
<td>**DepDelay**</td>
<td>Difference in minutes between scheduled and actual departure time; Early departures show negative numbers (target variable)</td>
<td>Int</td>
<tr>
<tr>
<td>DepDel15</td>
<td>Departure Delay Indicator, 15 Minutes or More (1=Yes)</td>
<td></td>
<tr>
<tr>
<td>DepartureDelayGroups</td>
<td>Departure Delay intervals, every (15 minutes from <-15 to >180)</td>
<td></td>
<tr>
<tr>
<td>DepTimeBlk</td>
<td>Scheduled Departure Time Block, Hourly Intervals</td>
<td></td>
<tr>
<tr>
<td>TaxiOut</td>
<td>Taxi Out Time, in Minutes</td>
<td></td>
<tr>
<tr>
<td>WheelsOff</td>
<td>Wheels Off Time (local time: hhmm)</td>
<td></td>
<tr>
<tr>
<td>Wheels On</td>
<td>Wheels On Time (local time: hhmm)</td>
<td></td>
<tr>
<tr>
<td>TaxiIn</td>
<td>Taxi In Time, in Minutes</td>
<td></td>
<tr>
<tr>
<td>CRSArrTime</td>
<td>Schduled Arrival Time (local time: hhmm)</td>
<td></td>
<tr>
<tr>
<td>ArrTime</td>
<td>Actual Arrival Time (local time: hhmm)</td>
<td></td>
<tr>
<tr>
<td>ArrDelay</td>
<td>Difference in minutes between scheduled and actual arrival time; Early arrivals show negative numbers</td>
<td></td>
<tr>
<tr>
<td>ArrDel15</td>
<td>Arrival Delay Indicator, 15 Minutes or More (1=Yes)</td>
<td></td>
<tr>
<tr>
<td>ArrivalDelayGroups</td>
<td>Arrival Delay intervals, every (15-minutes from <-15 to >180)</td>
<td></td>
<tr>
<tr>
<td>ArrTimeBlk</td>
<td>Scheduled Arrival Time Block, Hourly Intervals</td>
<td></td>
<tr>
<tr>
<td>CRSElapsedTime</td>
<td>CRS Elapsed Time of Flight, in Minutes</td>
<td></td>
<tr>
<tr>
<td>ActualElapsedTime</td>
<td>Elapsed Time of Flight, in Minutes</td>
<td></td>
<tr>
<tr>
<td>AirTime</td>
<td>Flight Time, in Minutes</td>
<td></td>
<tr>
<tr>
<td>Flights</td>
<td>Number of Flights</td>
<td></td>
<tr>
<tr>
<td>Distance</td>
<td>Distance between airports (miles)</td>
<td></td>
<tr>
<tr>
<td>DistanceGroup</td>
<td>Distance Intervals, every 250 Miles, for Flight Segment</td>
<td></td>
<tr>
<tr>
<td>CarrierDelay</td>
<td>Carrier Delay, in Minutes</td>
<td></td>
<tr>
<tr>
<td>WeatherDelay</td>
<td>Weather Delay, in Minutes</td>
<td></td>
<tr>
<tr>
<td>NASDelay</td>
<td>National Air System Delay, in Minutes</td>
<td></td>
<tr>
<tr>
<td>SecurityDelay</td>
<td>Security Delay, in Minutes</td>
<td></td>
<tr>
<tr>
<td>LateAircraftDelay</td>
<td>Late Aircraft Delay, in Minutes</td>
<td></td>
<tr>
<tr>
</table>

# 3 Feature Exploration, Feature Engeneering, Feature Encoding, Feature Selection and Data Frame Preparation

For this project four data frames will be constructed - data_1H_arr, data_1H_dep, data_1M_arr data_1M_dep.<br> data_1H_arr will contain all features relevant for arrival delays that are available one day before the predicted delay.<br> data_1H_dep will contain all features relevant for departure delays that are available one day before the predicted delay.<br> data_1M_arr will contain the features relevant for arrival delays which are available one month prior to the predicted delay.<br> data_1M_dep will contain the features relevant for departure delays which are available one month prior to the predicted delay.
<br> .loc is used, so the new data frame inherits the structure from the df data frame.

In [ ]:
data_1H_arr = data_1H_dep = data_1M_arr = data_1M_dep = data_6M_arr = data_6M_dep = df.loc[:,[]]

data_6M_arr and data_6M_dep will contain only the strategic features:

['Year', 'Month', 'DayOfMonth', 'DayOfWeek', 'FlightDate', 'Reporting_Airline',<br>
'Tail_Number', 'Flight_Number_Reporting_Airline', 'OriginAirportID', 'Origin',<br>
'DestAirportID', 'Dest', 'CRSDepTime', 'DepDelay', 'DepTimeBlk', 'CRSArrTime',<br>
'ArrDelay', 'ArrTimeBlk', 'CRSElapsedTime', 'Distance', 'DistanceGroup']

The departure and arrival features will be added accordingly.

Below, I will follow the same approach for every feature in our data cleansing process. I will first check for the correctness of the data and then bring it into an appropriate form in which an algorithm can find meaning. Lastly, we will add the engineered feature to our target datasets.

In [ ]:
# Define helper function for encoding
def encode_cyclical_feature(input_df, output_df, column, period):
    """ Encodes cyclical features to sine and cosine

    Args:
        input_df (DataFrame): The input data frame
        output_df (DataFrame): The output data frame
        column (str): The feature to encode
        period (int64): The cycle duration
    """
    sin_values = np.sin(2 * np.pi * input_df[column] / period)
    cos_values = np.cos(2 * np.pi * input_df[column] / period)

    output_df[column + '_sin'] = sin_values
    output_df[column + '_cos'] = cos_values

### Year

In [ ]:
print("----------Gaeneral feature information----------")
print(df.Year.describe())
print()
print("----------Value information----------")
print(df.Year.value_counts())

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='Year', bins=df['Year'].nunique(),
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
plt.xlabel('Year')
plt.ylabel('Number of Values')
plt.title('Number of Values for each Year')
plt.show()

In [ ]:
data_1H_arr['Year'] = data_1H_dep['Year'] = data_1M_arr['Year'] = data_1M_dep['Year'] = data_6M_arr['Year'] = data_6M_dep['Year'] = df['Year']

### Month

In [ ]:
print("----------General feature information----------")
print(df.groupby('Month').Month.describe())
print()
print("----------Value information----------")
print(df.groupby('Month').Month.value_counts())

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='Month', bins=df['Month'].nunique(),
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
plt.xlabel('Month')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each Month')
plt.show()

In [ ]:
encode_cyclical_feature(df, data_1H_arr, 'Month', 12)
encode_cyclical_feature(df, data_1H_dep, 'Month', 12)
encode_cyclical_feature(df, data_1M_arr, 'Month', 12)
encode_cyclical_feature(df, data_1M_dep, 'Month', 12)
encode_cyclical_feature(df, data_6M_arr, 'Month', 12)
encode_cyclical_feature(df, data_6M_dep, 'Month', 12)

### Season

In [ ]:
print("----------General feature information----------")
print(df.groupby('Season').Season.describe())
print()
print("----------Value information----------")
print(df.groupby('Season').Season.value_counts())

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='Season', bins=4,
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
# Get the x-axis ticks
#bin_edges = hist.get_xticks()
# Set the x-axis ticks to be at the edges of the bins
plt.xticks([1, 2, 3, 4])
plt.xlabel('Season')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each Season')
plt.show()

In [ ]:
data_1H_arr['Season'] = data_1H_dep['Season'] = data_1M_arr['Season'] = data_1M_dep['Season'] = data_6M_arr['Season'] = data_6M_dep['Season'] = df['Season']

### DayOfMonth

In [ ]:
print("----------General feature information----------")
print(df.groupby('DayOfMonth').DayOfMonth.describe())
print()
print("----------Value information----------")
print(df.groupby('DayOfMonth').DayOfMonth.value_counts())

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='DayOfMonth', bins=df['DayOfMonth'].nunique(),
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
plt.xlabel('Day of the Month')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each Day of the Month')
plt.show()

In [ ]:
encode_cyclical_feature(df, data_1H_arr, 'DayOfMonth', 31)
encode_cyclical_feature(df, data_1H_dep, 'DayOfMonth', 31)
encode_cyclical_feature(df, data_1M_arr, 'DayOfMonth', 31)
encode_cyclical_feature(df, data_1M_dep, 'DayOfMonth', 31)
encode_cyclical_feature(df, data_6M_arr, 'DayOfMonth', 31)
encode_cyclical_feature(df, data_6M_dep, 'DayOfMonth', 31)

### DayOfWeek

In [ ]:
print("----------General feature information----------")
print(df.groupby('DayOfWeek').DayOfWeek.describe())
print()
print("----------Value information----------")
print(df.groupby('DayOfWeek').DayOfWeek.value_counts())

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='DayOfWeek', bins=df['DayOfWeek'].nunique(),
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
plt.xlabel('Day of the Week')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each Day of the Week')
plt.show()

In [ ]:
encode_cyclical_feature(df, data_1H_arr, 'DayOfWeek', 7)
encode_cyclical_feature(df, data_1H_dep, 'DayOfWeek', 7)
encode_cyclical_feature(df, data_1M_arr, 'DayOfWeek', 7)
encode_cyclical_feature(df, data_1M_dep, 'DayOfWeek', 7)
encode_cyclical_feature(df, data_6M_arr, 'DayOfWeek', 7)
encode_cyclical_feature(df, data_6M_dep, 'DayOfWeek', 7)

In [ ]:
data_1H_arr.head()

### FlightDate

In [ ]:
print("----------General feature information----------")
print(df.groupby('FlightDate').FlightDate.describe())
print()
print("----------Value information----------")
print(df.groupby('FlightDate').FlightDate.value_counts())

### Reporting Airline

In [ ]:
print("----------General feature information----------")
print(df.groupby('Reporting_Airline').Reporting_Airline.describe())
print()
print("----------Value information----------")
print(df.groupby('Reporting_Airline').Reporting_Airline.value_counts())

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='Reporting_Airline', bins=df['Reporting_Airline'].nunique(),
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
plt.xlabel('Reporting_Airline')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each Reporting_Airline')
plt.show()

In [ ]:
bi_DF = preprocessing.LabelBinarizer()
bi_dummys = bi_DF.fit_transform(df['Reporting_Airline'])
print(bi_dummys.sum(axis=0))
print(df.Reporting_Airline.value_counts())

In [ ]:
print(df['Reporting_Airline'].dtype)

In [ ]:
# Assuming 'df' is your original DataFrame and 'AirlineCode' is the column to encode
df_encoded = pd.get_dummies(df[['Reporting_Airline']], prefix='Airline')

# Cast boolean columns to integers
df_encoded = df_encoded.astype(int)

# Assuming 'df_result' is another DataFrame where you want to save the encoded column
data_1H_arr = pd.concat([data_1H_arr, df_encoded], axis=1)
data_1H_dep = pd.concat([data_1H_dep, df_encoded], axis=1)
data_1M_arr = pd.concat([data_1M_arr, df_encoded], axis=1)
data_1M_dep = pd.concat([data_1M_dep, df_encoded], axis=1)
data_6M_arr = pd.concat([data_6M_arr, df_encoded], axis=1)
data_6M_dep = pd.concat([data_6M_dep, df_encoded], axis=1)

In [ ]:
print(df_encoded.dtypes)

In [ ]:
data_1M_dep.head()

### Tail Number

In [ ]:
print("----------General feature information----------")
print(df.groupby('Tail_Number').Tail_Number.describe())
print()
print("----------Value information----------")
print(df.groupby('Tail_Number').Tail_Number.value_counts())

In [ ]:
print(df.Tail_Number.nunique())

### Flight Number Reporting Airline

In [ ]:
print("----------General feature information----------")
print(df.groupby('Flight_Number_Reporting_Airline').Flight_Number_Reporting_Airline.describe())
print()
print("----------Value information----------")
print(df.groupby('Flight_Number_Reporting_Airline').Flight_Number_Reporting_Airline.value_counts())

In [ ]:
print(df.Flight_Number_Reporting_Airline.nunique())

### Origin Airport ID

In [ ]:
print("----------General feature information----------")
print(df.groupby('OriginAirportID').OriginAirportID.describe())
print()
print("----------Value information----------")
print(df.groupby('OriginAirportID').OriginAirportID.value_counts())

In [ ]:
print(df.OriginAirportID.nunique())

### Origin

In [ ]:
print("----------General feature information----------")
print(df.Origin.describe())
print()
print("----------Value information----------")
print(df.groupby('Origin').Origin.value_counts())

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='Origin', bins=df['Origin'].nunique(),
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
plt.xlabel('Origin')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each Origin')
plt.show()

In [ ]:
# Add the Origin to the result DataFrame
data_1H_arr['Origin'] = df['Origin'].copy()
data_1H_dep['Origin'] = df['Origin'].copy()
data_1M_arr['Origin'] = df['Origin'].copy()
data_1M_dep['Origin'] = df['Origin'].copy()
data_6M_arr['Origin'] = df['Origin'].copy()
data_6M_dep['Origin'] = df['Origin'].copy()

data_1H_arr.head()

In [ ]:
origin_counts = df['Origin'].value_counts()

# Sort the unique origins by the number of data points
sorted_origins = origin_counts.index.tolist()

# Print the sorted list of origins
print(sorted_origins)

# Create an ordinal mapping dictionary
ordinal_mapping = {code: i for i, code in enumerate(sorted_origins)}

# Add a new column with ordinal values to the DataFrame
data_1H_arr['Origin'] = data_1H_arr['Origin'].map(ordinal_mapping)
data_1H_dep['Origin'] = data_1H_dep['Origin'].map(ordinal_mapping)
data_1M_arr['Origin'] = data_1M_arr['Origin'].map(ordinal_mapping)
data_1M_dep['Origin'] = data_1M_dep['Origin'].map(ordinal_mapping)
data_6M_arr['Origin'] = data_6M_arr['Origin'].map(ordinal_mapping)
data_6M_dep['Origin'] = data_6M_dep['Origin'].map(ordinal_mapping)

In [ ]:
data_1H_arr.head()

In [ ]:
data_1H_arr.Origin.nunique()

In [ ]:
print(data_1H_arr.Origin.unique())

### OriginLongitude

In [ ]:
print("----------General feature information----------")
print(df.OriginLongitude.describe())
print()
print("----------Value information----------")
print(df.groupby('OriginLongitude').OriginLongitude.value_counts())

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='OriginLongitude', bins=df['OriginLongitude'].nunique(),
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
plt.xlabel('OriginLongitude')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each OriginLongitude')
plt.show()

In [ ]:
# Add the top origin lon to the result DataFrame
data_1H_arr['OriginLongitude'] = df['OriginLongitude'].copy()
data_1H_dep['OriginLongitude'] = df['OriginLongitude'].copy()
data_1M_arr['OriginLongitude'] = df['OriginLongitude'].copy()
data_1M_dep['OriginLongitude'] = df['OriginLongitude'].copy()
data_6M_arr['OriginLongitude'] = df['OriginLongitude'].copy()
data_6M_dep['OriginLongitude'] = df['OriginLongitude'].copy()

data_1H_arr.head()

In [ ]:
data_1H_arr.head()

### OriginLatitude

In [ ]:
print("----------General feature information----------")
print(df.OriginLatitude.describe())
print()
print("----------Value information----------")
print(df.groupby('OriginLatitude').OriginLatitude.value_counts())

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='OriginLatitude', bins=df['OriginLatitude'].nunique(),
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
plt.xlabel('OriginLatitude')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each OriginLatitude')
plt.show()

In [ ]:
# Add the top origin lon to the result DataFrame
data_1H_arr['OriginLatitude'] = df['OriginLatitude'].copy()
data_1H_dep['OriginLatitude'] = df['OriginLatitude'].copy()
data_1M_arr['OriginLatitude'] = df['OriginLatitude'].copy()
data_1M_dep['OriginLatitude'] = df['OriginLatitude'].copy()
data_6M_arr['OriginLatitude'] = df['OriginLatitude'].copy()
data_6M_dep['OriginLatitude'] = df['OriginLatitude'].copy()

data_1H_arr.head()

In [ ]:
data_1H_arr.head()

### Dest Airport ID

In [ ]:
print("----------General feature information----------")
print(df.DestAirportID.describe())
print()
print("----------Value information----------")
print(df.DestAirportID.value_counts())

In [ ]:
print(df.DestAirportID.nunique())

### Dest

In [ ]:
print("----------General feature information----------")
print(df.Dest.describe())
print()
print("----------Value information----------")
print(df.Dest.value_counts())

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='Dest', bins=df['Dest'].nunique(),
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
plt.xlabel('Dest')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each Dest')
plt.show()

In [ ]:
# Add the Dest to the result DataFrame
data_1H_arr['Dest'] = df['Dest'].copy()
data_1H_dep['Dest'] = df['Dest'].copy()
data_1M_arr['Dest'] = df['Dest'].copy()
data_1M_dep['Dest'] = df['Dest'].copy()
data_6M_arr['Dest'] = df['Dest'].copy()
data_6M_dep['Dest'] = df['Dest'].copy()

data_1H_arr.head()

In [ ]:
dest_counts = df['Dest'].value_counts()

# Sort the unique destinations by the number of data points
sorted_destinations = dest_counts.index.tolist()

# Print the sorted list of destinations
print(sorted_destinations)

# Create an ordinal mapping dictionary
ordinal_mapping = {code: i for i, code in enumerate(sorted_destinations)}

# Add a new column with ordinal values to the DataFrame
data_1H_arr['Dest'] = data_1H_arr['Dest'].map(ordinal_mapping)
data_1H_dep['Dest'] = data_1H_dep['Dest'].map(ordinal_mapping)
data_1M_arr['Dest'] = data_1M_arr['Dest'].map(ordinal_mapping)
data_1M_dep['Dest'] = data_1M_dep['Dest'].map(ordinal_mapping)
data_6M_arr['Dest'] = data_6M_arr['Dest'].map(ordinal_mapping)
data_6M_dep['Dest'] = data_6M_dep['Dest'].map(ordinal_mapping)

In [ ]:
data_1H_arr.head()

In [ ]:
data_1H_arr.Dest.nunique()

### DestLongitude

In [ ]:
print("----------General feature information----------")
print(df.DestLongitude.describe())
print()
print("----------Value information----------")
print(df.groupby('DestLongitude').DestLongitude.value_counts())

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='DestLongitude', bins=df['DestLongitude'].nunique(),
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
plt.xlabel('DestLongitude')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each DestLongitude')
plt.show()

In [ ]:
# Add the top origin lon to the result DataFrame
data_1H_arr['DestLongitude'] = df['DestLongitude'].copy()
data_1H_dep['DestLongitude'] = df['DestLongitude'].copy()
data_1M_arr['DestLongitude'] = df['DestLongitude'].copy()
data_1M_dep['DestLongitude'] = df['DestLongitude'].copy()
data_6M_arr['DestLongitude'] = df['DestLongitude'].copy()
data_6M_dep['DestLongitude'] = df['DestLongitude'].copy()

data_1H_arr.head()

In [ ]:
data_1H_arr.head()

### DestLatitude

In [ ]:
print("----------General feature information----------")
print(df.DestLatitude.describe())
print()
print("----------Value information----------")
print(df.groupby('DestLatitude').DestLatitude.value_counts())

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='DestLatitude', bins=df['DestLatitude'].nunique(),
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
plt.xlabel('DestLatitude')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each DestLatitude')
plt.show()

In [ ]:
# Add the top origin lon to the result DataFrame
data_1H_arr['DestLatitude'] = df['DestLatitude'].copy()
data_1H_dep['DestLatitude'] = df['DestLatitude'].copy()
data_1M_arr['DestLatitude'] = df['DestLatitude'].copy()
data_1M_dep['DestLatitude'] = df['DestLatitude'].copy()
data_6M_arr['DestLatitude'] = df['DestLatitude'].copy()
data_6M_dep['DestLatitude'] = df['DestLatitude'].copy()

data_1H_arr.head()

In [ ]:
data_1H_arr.head()

### CRSDepTime

In [ ]:
print("----------General feature information----------")
print(df.CRSDepTime.describe())
print()
print("----------Value information----------")
print(df.CRSDepTime.value_counts())
print("----------Sorted value information----------")
print(df.groupby('CRSDepTime').CRSDepTime.value_counts())

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='CRSDepTime', bins=df['CRSDepTime'].nunique(),
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
plt.xlabel('CRSDepTime')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each CRSDepTime')
plt.show()

In [ ]:
encode_cyclical_feature(df, data_1H_arr, 'CRSDepTime', 2359)
encode_cyclical_feature(df, data_1H_dep, 'CRSDepTime', 2359)
encode_cyclical_feature(df, data_1M_arr, 'CRSDepTime', 2359)
encode_cyclical_feature(df, data_1M_dep, 'CRSDepTime', 2359)
encode_cyclical_feature(df, data_6M_arr, 'CRSDepTime', 2359)
encode_cyclical_feature(df, data_6M_dep, 'CRSDepTime', 2359)

In [ ]:
data_1H_arr.head()

### DepTime

In [ ]:
print("----------General feature information----------")
print(df.DepTime.describe())
print()
print("----------Value information----------")
print(df.DepTime.value_counts())

### DepDelay

In [ ]:
print("----------General feature information----------")
print(df.DepDelay.describe())
print()
print("----------Value information----------")
print(df.DepDelay.value_counts())
print("----------Sorted value information----------")
print(df.groupby('DepDelay').DepDelay.value_counts())

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='DepDelay', bins=df['DepDelay'].nunique(),
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
plt.xlabel('DepDelay')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each DepDelay')
plt.xlim(right=180)
plt.show()

data = df.DepDelay
# Fit a probability distribution (Gaussian distribution) to the data
mu, std = norm.fit(data)

# Create a range of values for the x-axis
x = np.linspace(data.min(), data.max(), 100)

# Calculate the probability density function (PDF) for the fitted Gaussian distribution
pdf = norm.pdf(x, mu, std)

# Plot the data and the fitted distribution
plt.figure(figsize=(10, 5))
plt.hist(data, bins=df['DepDelay'].nunique(), density=True, alpha=0.6, color='b', label='Data')
plt.plot(x, pdf, 'r-', lw=2, label='Fitted Gaussian Distribution')
plt.title('Probability Distribution Prediction')
plt.xlabel('Value')
plt.ylabel('Probability')
plt.xlim(right=180)
plt.legend()
plt.show()

In [ ]:
min_value = df['DepDelay'].min()
max_value = df['DepDelay'].max()

# Display the result
print(f"The minimum value in the 'DepDelay' column is: {min_value}")
print(f"The minimum value in the 'DepDelay' column is: {max_value}")

According to the defintion, a delay above 180 minutes is considered as cancellation. Therefore delays above 180 minutes are not considered. This also reduces noise and removes outliers.

In [ ]:
# df_filtered = df[(df['DepDelay'] >= -30.0) & (df['DepDelay'] <= 180.0)]
# Exclude values outside the specified range
df_filtered_dep = df[df['DepDelay'] <= 180.0]

data_1H_dep = data_1H_dep.merge(df_filtered_dep[['DepDelay']], left_index=True, right_index=True, how='inner')
data_1M_dep = data_1M_dep.merge(df_filtered_dep[['DepDelay']], left_index=True, right_index=True, how='inner')
data_6M_dep = data_6M_dep.merge(df_filtered_dep[['DepDelay']], left_index=True, right_index=True, how='inner')

In [ ]:
# Count delays that are 15 minutes or larger
count_large_delays = len(df_filtered_dep[df_filtered_dep['DepDelay'] >= 15])

# Count delays less than 15 minutes
count_small_delays = len(df_filtered_dep[df_filtered_dep['DepDelay'] < 15])

print(f"Count of delays 15 minutes or larger: {count_large_delays}")
print(f"Count of delays less than 15 minutes: {count_small_delays}")

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=data_1H_dep, x='DepDelay', bins=data_1H_dep['DepDelay'].nunique(),
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
plt.xlabel('DepDelay')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each DepDelay')
plt.show()

data = data_1H_dep.DepDelay

# Fit a probability distribution (Gaussian distribution) to the data
mu, std = norm.fit(data)

# Create a range of values for the x-axis
x = np.linspace(data.min(), data.max(), 100)

# Calculate the probability density function (PDF) for the fitted Gaussian distribution
pdf = norm.pdf(x, mu, std)

# Plot the data and the fitted distribution
plt.figure(figsize=(10, 5))
plt.hist(data, bins=data_1H_dep['DepDelay'].nunique(), density=True, alpha=0.6, color='b', label='Data')
plt.plot(x, pdf, 'r-', lw=2, label='Fitted Gaussian Distribution')
plt.title('Probability Distribution Prediction')
plt.xlabel('Value')
plt.ylabel('Probability')
plt.legend()
plt.show()

Possibility use Log Transformation to decrease the right-skewness not given due to negative values.

In [ ]:
data_1H_dep.head()

In [ ]:
data_1H_dep.info()

### DepDel15

In [ ]:
print("----------General feature information----------")
print(df.DepDel15.describe())
print()
print("----------Value information----------")
print(df.DepDel15.value_counts())

### DepartureDelayGroups

In [ ]:
print("----------General feature information----------")
print(df.DepartureDelayGroups.describe())
print()
print("----------Value information----------")
print(df.groupby('DepartureDelayGroups').DepartureDelayGroups.value_counts())

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='DepartureDelayGroups', bins=14,
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
plt.xlabel('DepartureDelayGroups')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each DepartureDelayGroups')
plt.show()

### DepTimeBlk

In [ ]:
print("----------General feature information----------")
print(df.DepTimeBlk.describe())
print()
print("----------Value information----------")
print(df.groupby('DepTimeBlk').DepTimeBlk.value_counts())

# Sort the time blocks for plotting
sorted_time_blocks = df.groupby('DepTimeBlk').indices

plt.figure(figsize=(20, 6))
sns.countplot(data=df, x='DepTimeBlk', color="cornflowerblue", edgecolor=".3", linewidth=.5, order=sorted_time_blocks)
plt.xlabel('DepTimeBlk')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each DepTimeBlk')
plt.show()

In [ ]:
dep_time_block_mapping = {
    '0001-0559': 0,
    '0600-0659': 1,
    '0700-0759': 2,
    '0800-0859': 3,
    '0900-0959': 4,
    '1000-1059': 5,
    '1100-1159': 6,
    '1200-1259': 7,
    '1300-1359': 8,
    '1400-1459': 9,
    '1500-1559': 10,
    '1600-1659': 11,
    '1700-1759': 12,
    '1800-1859': 13,
    '1900-1959': 14,
    '2000-2059': 15,
    '2100-2159': 16,
    '2200-2259': 17,
    '2300-2359': 18
}

# Perform mapping
dep_time_blocks_encoded = df['DepTimeBlk'].map(dep_time_block_mapping)

# Concatenate the encoded columns to the result DataFrames
data_1H_arr = pd.concat([data_1H_arr, dep_time_blocks_encoded], axis=1)
data_1H_dep = pd.concat([data_1H_dep, dep_time_blocks_encoded], axis=1)
data_1M_arr = pd.concat([data_1M_arr, dep_time_blocks_encoded], axis=1)
data_1M_dep = pd.concat([data_1M_dep, dep_time_blocks_encoded], axis=1)
data_6M_arr = pd.concat([data_6M_arr, dep_time_blocks_encoded], axis=1)
data_6M_dep = pd.concat([data_6M_dep, dep_time_blocks_encoded], axis=1)

In [ ]:
data_1H_arr.info()

### TaxiOut

In [ ]:
print("----------General feature information----------")
print(df.TaxiOut.describe())
print()
print("----------Value information----------")
print(df.groupby('TaxiOut').TaxiOut.value_counts())

### WheelsOff

In [ ]:
print("----------General feature information----------")
print(df.WheelsOff.describe())
print()
print("----------Value information----------")
print(df.groupby('WheelsOff').WheelsOff.value_counts())

### WheelsOn

In [ ]:
print("----------General feature information----------")
print(df.WheelsOn.describe())
print()
print("----------Value information----------")
print(df.groupby('WheelsOn').WheelsOn.value_counts())

### TaxiIn

In [ ]:
print("----------General feature information----------")
print(df.TaxiIn.describe())
print()
print("----------Value information----------")
print(df.groupby('TaxiIn').TaxiIn.value_counts())

### CRSArrTime

In [ ]:
print("----------General feature information----------")
print(df.CRSArrTime.describe())
print()
print("----------Value information----------")
print(df.CRSArrTime.value_counts())
print()
print("----------Sorted value information----------")
print(df.groupby('CRSArrTime').CRSArrTime.value_counts())

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='CRSArrTime', bins=df['CRSArrTime'].nunique(),
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
plt.xlabel('CRSArrTime')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each CRSArrTime')
plt.show()

In [ ]:
encode_cyclical_feature(df, data_1H_arr, 'CRSArrTime', 2359)
encode_cyclical_feature(df, data_1H_dep, 'CRSArrTime', 2359)
encode_cyclical_feature(df, data_1M_arr, 'CRSArrTime', 2359)
encode_cyclical_feature(df, data_1M_dep, 'CRSArrTime', 2359)
encode_cyclical_feature(df, data_6M_arr, 'CRSArrTime', 2359)
encode_cyclical_feature(df, data_6M_dep, 'CRSArrTime', 2359)

In [ ]:
data_1H_arr.head()

### ArrDelay

In [ ]:
print("----------General feature information----------")
print(df.ArrDelay.describe())
print()
print("----------Value information----------")
print(df.ArrDelay.value_counts())
print("----------Sorted value information----------")
print(df.groupby('ArrDelay').ArrDelay.value_counts())

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='ArrDelay', bins=df['ArrDelay'].nunique(),
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
plt.xlabel('ArrDelay')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each ArrDelay')
plt.xlim(right=180)
plt.show()

data = df.ArrDelay
# Fit a probability distribution (Gaussian distribution) to the data
mu, std = norm.fit(data)

# Create a range of values for the x-axis
x = np.linspace(data.min(), data.max(), 100)

# Calculate the probability density function (PDF) for the fitted Gaussian distribution
pdf = norm.pdf(x, mu, std)

# Plot the data and the fitted distribution
plt.figure(figsize=(10, 5))
plt.hist(data, bins=df['ArrDelay'].nunique(), density=True, alpha=0.6, color='b', label='Data')
plt.plot(x, pdf, 'r-', lw=2, label='Fitted Gaussian Distribution')
plt.title('Probability Distribution Prediction')
plt.xlabel('Value')
plt.ylabel('Probability')
plt.xlim(right=180)
plt.legend()
plt.show()

In [ ]:
min_value = df['ArrDelay'].min()
max_value = df['ArrDelay'].max()

# Display the result
print(f"The minimum value in the 'ArrDelay' column is: {min_value}")
print(f"The minimum value in the 'ArrDelay' column is: {max_value}")

In [ ]:
df_filtered_arr = df[df['ArrDelay'] <= 180.0]

data_1H_arr = data_1H_arr.merge(df_filtered_arr[['ArrDelay']], left_index=True, right_index=True, how='inner')
data_1M_arr = data_1M_arr.merge(df_filtered_arr[['ArrDelay']], left_index=True, right_index=True, how='inner')
data_6M_arr = data_6M_arr.merge(df_filtered_arr[['ArrDelay']], left_index=True, right_index=True, how='inner')

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(data=data_1H_arr, x='ArrDelay', bins=data_1H_arr['ArrDelay'].nunique(),
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
plt.xlabel('ArrDelay')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each ArrDelay')
plt.show()

data = data_1H_arr.ArrDelay

# Fit a probability distribution (Gaussian distribution) to the data
mu, std = norm.fit(data)

# Create a range of values for the x-axis
x = np.linspace(data.min(), data.max(), 100)

# Calculate the probability density function (PDF) for the fitted Gaussian distribution
pdf = norm.pdf(x, mu, std)

# Plot the data and the fitted distribution
plt.figure(figsize=(10, 5))
plt.hist(data, bins=data_1H_arr['ArrDelay'].nunique(), density=True, alpha=0.6, color='b', label='Data')
plt.plot(x, pdf, 'r-', lw=2, label='Fitted Gaussian Distribution')
plt.title('Probability Distribution Prediction')
plt.xlabel('Value')
plt.ylabel('Probability')
plt.legend()
plt.show()

In [ ]:
data_1H_arr.info()

In [ ]:
data_1H_arr.head()

### ArrivalDelayGroups

In [ ]:
print("----------General feature information----------")
print(df.ArrivalDelayGroups.describe())
print()
print("----------Value information----------")
print(df.groupby('ArrivalDelayGroups').ArrivalDelayGroups.value_counts())

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='ArrivalDelayGroups', bins=14,
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
plt.xlabel('ArrivalDelayGroups')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each ArrivalDelayGroups')
plt.show()

### ArrTimeBlk

In [ ]:
print("----------General feature information----------")
print(df.ArrTimeBlk.describe())
print()
print("----------Value information----------")
print(df.groupby('ArrTimeBlk').ArrTimeBlk.value_counts())

# Sort the time blocks for plotting
sorted_time_blocks = df.groupby('ArrTimeBlk').indices

plt.figure(figsize=(20, 6))
sns.countplot(data=df, x='ArrTimeBlk', color="cornflowerblue", edgecolor=".3", linewidth=.5, order=sorted_time_blocks)
plt.xlabel('ArrTimeBlk')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each ArrTimeBlk')
plt.show()

In [ ]:
print(df.groupby('ArrTimeBlk').ArrTimeBlk.unique())

In [ ]:
arr_time_block_mapping = {
    '0001-0559': 0,
    '0600-0659': 1,
    '0700-0759': 2,
    '0800-0859': 3,
    '0900-0959': 4,
    '1000-1059': 5,
    '1100-1159': 6,
    '1200-1259': 7,
    '1300-1359': 8,
    '1400-1459': 9,
    '1500-1559': 10,
    '1600-1659': 11,
    '1700-1759': 12,
    '1800-1859': 13,
    '1900-1959': 14,
    '2000-2059': 15,
    '2100-2159': 16,
    '2200-2259': 17,
    '2300-2359': 18
}

# Perform mapping
arr_time_blocks_encoded = df['ArrTimeBlk'].map(arr_time_block_mapping)

# Concatenate the encoded columns to the original DataFrame
data_1H_arr = pd.concat([data_1H_arr, arr_time_blocks_encoded], axis=1)
data_1H_dep = pd.concat([data_1H_dep, arr_time_blocks_encoded], axis=1)
data_1M_arr = pd.concat([data_1M_arr, arr_time_blocks_encoded], axis=1)
data_1M_dep = pd.concat([data_1M_dep, arr_time_blocks_encoded], axis=1)
data_6M_arr = pd.concat([data_6M_arr, arr_time_blocks_encoded], axis=1)
data_6M_dep = pd.concat([data_6M_dep, arr_time_blocks_encoded], axis=1)

In [ ]:
data_1H_arr.info()

### CRSElapsedTime

In [ ]:
print("----------General feature information----------")
print(df.CRSElapsedTime.describe())
print()
print("----------Value information----------")
print(df.CRSElapsedTime.value_counts())
print()
print("----------Sorted value information----------")
print(df.groupby('CRSElapsedTime').CRSElapsedTime.value_counts())

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='CRSElapsedTime', bins=df['CRSElapsedTime'].nunique(),
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
plt.xlabel('CRSElapsedTime')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each CRSElapsedTime')
plt.show()

In [ ]:
data_1H_arr['CRSElapsedTime'] = data_1H_dep['CRSElapsedTime'] = data_1M_arr['CRSElapsedTime'] = data_1M_dep['CRSElapsedTime'] = data_6M_arr['CRSElapsedTime'] = data_6M_dep['CRSElapsedTime'] = df['CRSElapsedTime']
data_1H_arr.info()

### ActualElapsedTime

In [ ]:
print("----------General feature information----------")
print(df.ActualElapsedTime.describe())
print()
print("----------Value information----------")
print(df.ActualElapsedTime.value_counts())
print()
print("----------Sorted value information----------")
print(df.groupby('ActualElapsedTime').ActualElapsedTime.value_counts())

### AirTime

In [ ]:
print("----------General feature information----------")
print(df.AirTime.describe())
print()
print("----------Value information----------")
print(df.AirTime.value_counts())
print()
print("----------Sorted value information----------")
print(df.groupby('AirTime').AirTime.value_counts())

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='AirTime', bins=df['AirTime'].nunique(),
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
plt.xlabel('AirTime')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each AirTime')
plt.show()

### Distance

In [ ]:
print("----------General feature information----------")
print(df.Distance.describe())
print()
print("----------Value information----------")
print(df.Distance.value_counts())
print()
print("----------Sorted value information----------")
print(df.groupby('Distance').Distance.value_counts())

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='Distance', bins=df['Distance'].nunique(),
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
plt.xlabel('Distance')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each Distance')
plt.show()

In [ ]:
data_1H_arr['Distance'] = data_1H_dep['Distance'] = data_1M_arr['Distance'] = data_1M_dep['Distance'] = data_6M_arr['Distance'] = data_6M_dep['Distance'] = df['Distance']
data_1H_arr.info()

### DistanceGroup

In [ ]:
print("----------General feature information----------")
print(df.DistanceGroup.describe())
print()
print("----------Value information----------")
print(df.DistanceGroup.value_counts())
print()
print("----------Sorted value information----------")
print(df.groupby('DistanceGroup').DistanceGroup.value_counts())

plt.figure(figsize=(10, 6))
sns.histplot(data=df, x='DistanceGroup', bins=10,
             stat='count', palette="light:m_r", edgecolor=".3", linewidth=.5)
plt.xlabel('DistanceGroup')
plt.ylabel('Number of Values')
plt.title('Number of Values for Each DistanceGroup')
plt.show()

In [ ]:
data_1H_arr['DistanceGroup'] = data_1H_dep['DistanceGroup'] = data_1M_arr['DistanceGroup'] = data_1M_dep['DistanceGroup'] = data_6M_arr['DistanceGroup'] = data_6M_dep['DistanceGroup'] = df['DistanceGroup']
data_1H_arr.info()

### Weather

In [ ]:
hourly_weather_dep = ['HourlyAvgTemperatureDep',
                      'HourlyAvgDewPointDep',
                      'HourlyAvgRelativeHumidityDep',
                      'HourlyAvgWindSpeedDep',
                      'HourlyAvgVisibilityDep',]

hourly_weather_arr = ['HourlyAvgTemperatureArr',
                      'HourlyAvgDewPointArr',
                      'HourlyAvgRelativeHumidityArr',
                      'HourlyAvgWindSpeedArr',
                      'HourlyAvgVisibilityArr']

monthly_weather = ['MonthlyAvgTemperature',
                   'MonthlyAvgDewPoint',
                   'MonthlyAvgRelativeHumidity']

In [ ]:
for metric in hourly_weather_dep:
    data_1H_dep[metric] = df[metric]

for metric in hourly_weather_arr:
    data_1H_arr[metric] = df[metric]

for metric in monthly_weather:
    data_1M_dep[metric] = df[metric]
    data_1M_arr[metric] = df[metric]

In [ ]:
data_1H_arr.head()

In [ ]:
data_1H_arr.info()

### Separate in Arrival and Departure Flights

In [ ]:
print("Origin in 1H_dep:", data_1H_dep.Origin.nunique())
print("Dest in 1H_arr", data_1H_arr.Dest.nunique())

In [ ]:
# Separate arriving and departing flights using boolean indexing
data_1H_arr = data_1H_arr[data_1H_arr['Dest'] == 0]
data_1H_dep = data_1H_dep[data_1H_dep['Origin'] == 0]
data_1M_arr = data_1M_arr[data_1M_arr['Dest'] == 0]
data_1M_dep = data_1M_dep[data_1M_dep['Origin'] == 0]
data_6M_arr = data_6M_arr[data_6M_arr['Dest'] == 0]
data_6M_dep = data_6M_dep[data_6M_dep['Origin'] == 0]

In [ ]:
print("Origin in 1H_dep:", data_1H_dep.Origin.nunique())
print("Dest in 1H_arr", data_1H_arr.Dest.nunique())

In [ ]:
columns_to_drop = ['Dest', 'DestLongitude', 'DestLatitude']
data_1H_arr.drop(columns=columns_to_drop, inplace=True)
data_1M_arr.drop(columns=columns_to_drop, inplace=True)
data_6M_arr.drop(columns=columns_to_drop, inplace=True)

In [ ]:
columns_to_drop = ['Origin', 'OriginLongitude', 'OriginLatitude']
data_1H_dep.drop(columns=columns_to_drop, inplace=True)
data_1M_dep.drop(columns=columns_to_drop, inplace=True)
data_6M_dep.drop(columns=columns_to_drop, inplace=True)

In [ ]:
data_1H_arr.head()

In [ ]:
data_1H_dep.head()

In [ ]:
print(data_1H_dep.shape)

In [ ]:
print(data_1H_arr.shape)

## Multivariable Exploration

In [ ]:
datasets = [data_1H_arr, data_1H_dep, data_1M_arr, data_1M_dep, data_6M_arr, data_6M_dep]

In [ ]:
for dataset in datasets:
    print(dataset.isnull().sum())

In [ ]:
# Drop NaN values since they make up 1% of the data
data_1H_arr.dropna(inplace=True)
data_1H_dep.dropna(inplace=True)
data_1M_arr.dropna(inplace=True)
data_1M_dep.dropna(inplace=True)
data_6M_arr.dropna(inplace=True)
data_6M_dep.dropna(inplace=True)

### Pearson Correlation Matrix

In [ ]:
# Features to exclude from correlation analysis
exclude_features = ["ArrDelay", "DepDelay"] # Target variables should never be excluded
corr_features = []

In [ ]:
# Define the substring to identify Longitude and Latitude features
longitude_substring = 'Longitude'
latitude_substring = 'Latitude'

for idx, dataset in enumerate(datasets):
    # Exclude specified features from the correlation analysis
    dataset_subset = dataset.drop(columns=exclude_features, errors='ignore')

    correlation_matrix = dataset_subset.corr()
    correlated_features = set()
    correlated_pairs = []
    processed_features = set()

    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > 0.8:
                feature_1 = correlation_matrix.columns[i]
                feature_2 = correlation_matrix.columns[j]
                colname = correlation_matrix.columns[i]
                correlated_pairs.append((feature_1, feature_2, correlation_matrix.iloc[i, j]))

                # Only remove sine and cosine as well as longitude and latitude features if both correlate above the threshold with the same feature
                if ' (sin)' in feature_1:
                    cosine_feature = feature_1.replace(' (sin)', ' (cos)')
                    if cosine_feature in correlation_matrix.columns and cosine_feature not in processed_features:
                        colname_cosine = feature_1.replace(' (sin)', '')
                        correlated_pairs.append((feature_1, cosine_feature, correlation_matrix.loc[feature_1, cosine_feature]))
                        correlated_features.add(colname)
                        correlated_features.add(colname_cosine)
                        processed_features.add(colname)
                        processed_features.add(colname_cosine)
                    else:
                        correlated_features.add(colname)
                        processed_features.add(colname)
                elif ' (cos)' in feature_1 and feature_1 not in processed_features:
                    sin_feature = feature_1.replace(' (cos)', ' (sin)')
                    if sin_feature in correlation_matrix.columns:
                        colname_sin = sin_feature.replace(' (sin)', '')
                        correlated_pairs.append((sin_feature, feature_1, correlation_matrix.loc[sin_feature, feature_1]))
                        correlated_features.add(colname_sin)
                        correlated_features.add(colname)
                        processed_features.add(colname_sin)
                        processed_features.add(colname)
                    else:
                        correlated_features.add(colname)
                        processed_features.add(colname)
                elif longitude_substring in feature_1:
                    latitude_feature = feature_1.replace(longitude_substring, latitude_substring)
                    if latitude_feature in correlation_matrix.columns and latitude_feature not in processed_features:
                        correlated_pairs.append((feature_1, latitude_feature, correlation_matrix.loc[feature_1, latitude_feature]))
                        correlated_features.add(colname)
                        correlated_features.add(latitude_feature.replace(latitude_substring, ''))
                        processed_features.add(colname)
                        processed_features.add(latitude_feature)
                    else:
                        correlated_features.add(colname)
                        processed_features.add(colname)
                else:
                    correlated_features.add(colname)
                    processed_features.add(colname)

    print(correlated_features)
    print(correlated_pairs)
    corr_features.append(list(correlated_features))

    # Set figsize based on the index of the dataset
    figsize = (15, 21)

    # Create a heatmap
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    fig, ax = plt.subplots(figsize=figsize, dpi=300)

    # Create a heatmap without annotations
    heatmap = sns.heatmap(correlation_matrix, mask=mask, ax=ax, cmap='coolwarm', linewidths=.5, annot=False, vmin=-1, vmax=1, cbar_kws={})
    cbar = heatmap.collections[0].colorbar
    cbar.ax.tick_params(labelsize=16)

    # Annotate only the correlated feature pairs within the matrix
    for pair in correlated_pairs:
        i = np.where(correlation_matrix.columns == pair[0])[0][0]
        j = np.where(correlation_matrix.columns == pair[1])[0][0]
        ax.text(j + 0.5, i + 0.5, f"{pair[2]:.2f}", ha='center', va='center', fontsize=12, color='white', rotation='vertical')

    # ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)

    plt.show()

### Feature Selection

In [ ]:
print(len(corr_features))

In [ ]:
print(corr_features)

In [ ]:
# Remove non-existent feature
for sublist in corr_features:
    if 'CRSArrTime' in sublist:
        sublist.remove('CRSArrTime')

In [ ]:
print(corr_features)

In [ ]:
data_1H_arr = data_1H_arr.drop(corr_features[0], axis=1)
data_1H_dep = data_1H_dep.drop(corr_features[1], axis=1)
data_1M_arr = data_1M_arr.drop(corr_features[2], axis=1)
data_1M_dep = data_1M_dep.drop(corr_features[3], axis=1)
data_6M_arr = data_6M_arr.drop(corr_features[4], axis=1)
data_6M_dep = data_6M_dep.drop(corr_features[5], axis=1)

In [ ]:
cleaned_datasets = [data_1H_arr, data_1H_dep, data_1M_arr, data_1M_dep, data_6M_arr, data_6M_dep]

Check if the cleaned datasets still contain unallowed correlation pairs.

In [ ]:
# Define the substring to identify Longitude and Latitude features
longitude_substring = 'Longitude'
latitude_substring = 'Latitude'

for idx, dataset in enumerate(cleaned_datasets):
    # Exclude specified features from the correlation analysis
    dataset_subset = dataset.drop(columns=exclude_features, errors='ignore')

    correlation_matrix = dataset_subset.corr()
    correlated_features = set()
    correlated_pairs = []
    processed_features = set()

    for i in range(len(correlation_matrix.columns)):
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > 0.8:
                feature_1 = correlation_matrix.columns[i]
                feature_2 = correlation_matrix.columns[j]
                colname = correlation_matrix.columns[i]
                correlated_pairs.append((feature_1, feature_2, correlation_matrix.iloc[i, j]))

                # Only remove sine and cosine as well as longitude and latitude features if both correlate above the threshold with the same feature
                if ' (sin)' in feature_1:
                    cosine_feature = feature_1.replace(' (sin)', ' (cos)')
                    if cosine_feature in correlation_matrix.columns and cosine_feature not in processed_features:
                        colname_cosine = feature_1.replace(' (sin)', '')
                        correlated_pairs.append((feature_1, cosine_feature, correlation_matrix.loc[feature_1, cosine_feature]))
                        correlated_features.add(colname)
                        correlated_features.add(colname_cosine)
                        processed_features.add(colname)
                        processed_features.add(colname_cosine)
                    else:
                        correlated_features.add(colname)
                        processed_features.add(colname)
                elif ' (cos)' in feature_1 and feature_1 not in processed_features:
                    sin_feature = feature_1.replace(' (cos)', ' (sin)')
                    if sin_feature in correlation_matrix.columns:
                        colname_sin = sin_feature.replace(' (sin)', '')
                        correlated_pairs.append((sin_feature, feature_1, correlation_matrix.loc[sin_feature, feature_1]))
                        correlated_features.add(colname_sin)
                        correlated_features.add(colname)
                        processed_features.add(colname_sin)
                        processed_features.add(colname)
                    else:
                        correlated_features.add(colname)
                        processed_features.add(colname)
                elif longitude_substring in feature_1:
                    latitude_feature = feature_1.replace(longitude_substring, latitude_substring)
                    if latitude_feature in correlation_matrix.columns and latitude_feature not in processed_features:
                        correlated_pairs.append((feature_1, latitude_feature, correlation_matrix.loc[feature_1, latitude_feature]))
                        correlated_features.add(colname)
                        correlated_features.add(latitude_feature.replace(latitude_substring, ''))
                        processed_features.add(colname)
                        processed_features.add(latitude_feature)
                    else:
                        correlated_features.add(colname)
                        processed_features.add(colname)
                else:
                    correlated_features.add(colname)
                    processed_features.add(colname)

    print(correlated_features)
    print(correlated_pairs)
    corr_features.append(list(correlated_features))

    # Set figsize based on the index of the dataset
    figsize = (15, 21)

    # Create a heatmap
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    fig, ax = plt.subplots(figsize=figsize, dpi=300)

    # Create a heatmap without annotations
    heatmap = sns.heatmap(correlation_matrix, mask=mask, ax=ax, cmap='coolwarm', linewidths=.5, annot=False, vmin=-1, vmax=1, cbar_kws={})
    cbar = heatmap.collections[0].colorbar
    cbar.ax.tick_params(labelsize=16)

    # Annotate only the correlated feature pairs within the matrix
    for pair in correlated_pairs:
        i = np.where(correlation_matrix.columns == pair[0])[0][0]
        j = np.where(correlation_matrix.columns == pair[1])[0][0]
        ax.text(j + 0.5, i + 0.5, f"{pair[2]:.2f}", ha='center', va='center', fontsize=12, color='white', rotation='vertical')

    # ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    plt.xticks(fontsize=18)
    plt.yticks(fontsize=18)

    plt.show()

## Sampling

In [ ]:
arrivals_data = [data_1H_arr, data_1M_arr, data_6M_arr]
departures_data = [data_1H_dep, data_1M_dep, data_6M_dep]
sampled_arrivals = []
sampled_departures = []

In [ ]:
# Define the common bins for 'DepDelay' stratification
bins = [-float('inf'), -75, -60, -45, -30, -15, 0, 15, 30, 45, 60, 75, 90, 105, 120, 135, 150, 165, float('inf')]

# Define the maximum fraction to keep
max_fraction_to_keep = 0.2

# Loop through arrivals_data
for arrival in arrivals_data:
    # Stratified sampling based on 'ArrDelay'
    strata = pd.cut(arrival['ArrDelay'], bins=bins)
    proportions = strata.value_counts(normalize=True)

    # Calculate the maximum allowed fraction based on the maximum limit
    max_allowed_fraction = proportions * max_fraction_to_keep

    # Sample from each stratum with the specified fraction, ensuring it doesn't exceed the maximum
    sampled_data = arrival.groupby(strata, group_keys=False).apply(lambda x: x.sample(frac=min(max_allowed_fraction.loc[x.name], 1.0)))
    sampled_arrivals.append(sampled_data)

# Loop through departures_data
for departure in departures_data:
    # Stratified sampling based on 'DepDelay'
    strata = pd.cut(departure['DepDelay'], bins=bins)
    proportions = strata.value_counts(normalize=True)

    # Calculate the maximum allowed fraction based on the maximum limit
    max_allowed_fraction = proportions * max_fraction_to_keep

    # Sample from each stratum with the specified fraction
    sampled_data = departure.groupby(strata, group_keys=False).apply(lambda x: x.sample(frac=min(max_allowed_fraction.loc[x.name], 1.0)))
    sampled_departures.append(sampled_data)

In [ ]:
data_1H_arr = sampled_arrivals[0]
data_1H_dep = sampled_departures[0]
data_1M_arr = sampled_arrivals[1]
data_1M_dep = sampled_departures[1]
data_6M_arr = sampled_arrivals[2]
data_6M_dep = sampled_departures[2]

# 4 Splitting the Dataframes

### Split into x and y

In [ ]:
data_1H_arr_x, data_1H_dep_x, data_1M_arr_x, data_1M_dep_x, data_6M_arr_x, data_6M_dep_x = data_1H_arr.drop('ArrDelay', axis = 1), data_1H_dep.drop('DepDelay', axis = 1), data_1M_arr.drop('ArrDelay', axis = 1), data_1M_dep.drop('DepDelay', axis = 1), data_6M_arr.drop('ArrDelay', axis = 1), data_6M_dep.drop('DepDelay', axis = 1)

In [ ]:
data_1H_arr_y = data_1H_arr['ArrDelay']
data_1M_arr_y = data_1M_arr['ArrDelay']
data_6M_arr_y = data_6M_arr['ArrDelay']
data_1H_dep_y = data_1H_dep['DepDelay']
data_1M_dep_y = data_1M_dep['DepDelay']
data_6M_dep_y = data_6M_dep['DepDelay']

In [ ]:
data_1H_arr_x.info()

# 5 Scaling

In [ ]:
nscaler = preprocessing.MinMaxScaler()
data_1H_arr_x.iloc[:,:], data_1H_dep_x.iloc[:,:], data_1M_arr_x.iloc[:,:], data_1M_dep_x.iloc[:,:], data_6M_arr_x.iloc[:,:], data_6M_dep_x.iloc[:,:] = nscaler.fit_transform(data_1H_arr_x.iloc[:,:]), nscaler.fit_transform(data_1H_dep_x.iloc[:,:]), nscaler.fit_transform(data_1M_arr_x.iloc[:,:]), nscaler.fit_transform(data_1M_dep_x.iloc[:,:]), nscaler.fit_transform(data_6M_arr_x.iloc[:,:]), nscaler.fit_transform(data_6M_dep_x.iloc[:,:])

data_1H_arr_x.describe()

In [ ]:
data_1H_arr_x.head()

In [ ]:
data_1H_arr_x.info()

In [ ]:
print(data_1H_arr_x.isnull().sum())

In [ ]:
print(data_1H_arr_x.dtypes)

# 6 Partitioning

In [ ]:
training_set_percentage = 0.8 #@param {type:"slider", min:0, max:1, step:0.01}
X_1_train, X_1_test, Y_1_train, Y_1_test = train_test_split(data_1H_arr_x, data_1H_arr_y, test_size = 1 - training_set_percentage, random_state=0)
X_2_train, X_2_test, Y_2_train, Y_2_test = train_test_split(data_1H_dep_x, data_1H_dep_y, test_size = 1 - training_set_percentage, random_state=0)
X_3_train, X_3_test, Y_3_train, Y_3_test = train_test_split(data_1M_arr_x, data_1M_arr_y, test_size = 1 - training_set_percentage, random_state=0)
X_4_train, X_4_test, Y_4_train, Y_4_test = train_test_split(data_1M_dep_x, data_1M_dep_y, test_size = 1 - training_set_percentage, random_state=0)
X_5_train, X_5_test, Y_5_train, Y_5_test = train_test_split(data_6M_arr_x, data_6M_arr_y, test_size = 1 - training_set_percentage, random_state=0)
X_6_train, X_6_test, Y_6_train, Y_6_test = train_test_split(data_6M_dep_x, data_6M_dep_y, test_size = 1 - training_set_percentage, random_state=0)

Y_1_train_mean,Y_2_train_mean,Y_3_train_mean,Y_4_train_mean,Y_5_train_mean,Y_6_train_mean = Y_1_test.mean(),Y_2_test.mean(),Y_3_test.mean(),Y_4_test.mean(),Y_5_test.mean(),Y_6_test.mean()
Y_1_train_meandev,Y_2_train_meandev,Y_3_train_meandev,Y_4_train_meandev,Y_5_train_meandev,Y_6_train_meandev = sum((Y_1_train-Y_1_train_mean)**2),sum((Y_2_train-Y_2_train_mean)**2),sum((Y_3_train-Y_3_train_mean)**2),sum((Y_4_train-Y_4_train_mean)**2),sum((Y_5_train-Y_5_train_mean)**2),sum((Y_6_train-Y_6_train_mean)**2)
Y_1_test_meandev,Y_2_test_meandev,Y_3_test_meandev,Y_4_test_meandev,Y_5_test_meandev,Y_6_test_meandev= sum((Y_1_test-Y_1_train_mean)**2),sum((Y_2_test-Y_2_train_mean)**2),sum((Y_3_test-Y_3_train_mean)**2),sum((Y_4_test-Y_4_train_mean)**2),sum((Y_5_test-Y_5_train_mean)**2),sum((Y_6_test-Y_6_train_mean)**2)

Create a report dataframe to store the results.

In [ ]:
report = pd.DataFrame(columns=['Model', 'R2', 'Pseudo-R2', 'MeanTestScore', 'StdTestScore', 'MAE', 'MSE', 'RMSE', 'CRPS', 'Dataset'])

Create a list of datasets to be able to iterate through them quickly.

In [ ]:
datasets = [[X_1_train, Y_1_train, X_1_test, Y_1_test,Y_1_train_meandev,Y_1_test_meandev, '1H_arr', 0],
            [X_2_train, Y_2_train, X_2_test, Y_2_test,Y_2_train_meandev,Y_2_test_meandev, '1H_dep', 1],
            [X_3_train, Y_3_train, X_3_test, Y_3_test,Y_3_train_meandev,Y_3_test_meandev, '1M_arr', 2],
            [X_4_train, Y_4_train, X_4_test, Y_4_test,Y_4_train_meandev,Y_4_test_meandev, '1M_dep', 3],
            [X_5_train, Y_5_train, X_5_test, Y_5_test,Y_5_train_meandev,Y_5_test_meandev, '6M_arr', 4],
            [X_6_train, Y_6_train, X_6_test, Y_6_test,Y_6_train_meandev,Y_6_test_meandev, '6M_dep', 5]]

# 7 Algorithms

### Styling

In [ ]:
blue_shades = ['#ADD8E6', '#4682B4', '#000080']
green_shades = ['#90EE90', '#008000', '#006400']
orange_shades = ['#FFD700', '#FFA500', '#FF8C00']
red_shades = ['#FFA07A', '#CD5C5C', '#8B0000']

# Plotting colors
colors = [blue_shades, green_shades, orange_shades, red_shades]
dark_gray = '#A9A9A9'

### Metrics

In [ ]:
def calculate_cdf(predictions):
    sorted_predictions = np.sort(predictions)
    cdf = np.arange(1, len(sorted_predictions) + 1) / len(sorted_predictions)
    return cdf

def calculate_crps(predicted_cdf, observed_values):
    crps_values = ps.crps_ensemble(observed_values, predicted_cdf)
    return crps_values

# Define a scorer for CRPS
def crps_scorer(model, X, y):
    # Make predictions with the model
    predictions = model.predict(X)

    # Calculate CRPS for each sample in the test set
    crps_values = calculate_crps(calculate_cdf(predictions), y)

    # Return the mean CRPS as the score
    return np.mean(crps_values)

## Random Forrest

In [ ]:
from scipy.stats import gaussian_kde
for X_train, Y_train, X_test, Y_test, Y_train_meandev, Y_test_meandev, data_set, data_set_id in datasets:
    print("-------- Data Set:", data_set, "--------")
    # Define the random forest regressor
    RForregCV = RandomForestRegressor(random_state=0)

    # Define the parameter grid for grid search
    param_grid = {
        'max_depth': [8], # tested: [5, 8, 9, 10, 11, 12, 15, 20, 25]
        'n_estimators': [250] # tested: [10, 50, 100, 150, 200, 250, 300, 400, 500]
    }

    xlabel = ''
    color = 'green'
    lightorange = '#ffcc66'

    if data_set_id % 2 == 0:
        xlabel = 'Arrival delay - STD at 0 [min]'
        color = 'lightblue'
    else:
        xlabel = 'Departure delay - STA at 0 [min]'
        color = lightorange

    # Perform grid search with cross-validation
    CV_rfmodel = GridSearchCV(estimator=RForregCV, param_grid=param_grid, cv=10, scoring=crps_scorer)
    CV_rfmodel.fit(X_train, Y_train)
    print("Best Parameters:", CV_rfmodel.best_params_)

    # Set the model with the best parameters
    RForregCV = RForregCV.set_params(**CV_rfmodel.best_params_)

    # Fit the model on the training data
    RForregCV.fit(X_train, Y_train)

    # Get predictions from individual trees
    tree_predictions = [tree.predict(X_test) for tree in RForregCV.estimators_]

    # Combine predictions by taking the average
    combined_predictions = np.mean(np.vstack(tree_predictions), axis=0)

    kde = gaussian_kde(combined_predictions.T)
    # Evaluate the density at specific points
    x_grid = np.linspace(min(combined_predictions.flatten()), max(combined_predictions.flatten()), 100)
    density_values = kde.evaluate(x_grid)

    # Generate a PDF for visualization
    x_values = np.linspace(min(combined_predictions.flatten()), max(combined_predictions.flatten()), 100)
    pdf_values = kde.evaluate(x_values)

    # Convert Y_test to a NumPy array and then create a KDE
    Y_test_array = np.array(Y_test)
    kde_test = gaussian_kde(Y_test_array.T)

    # Evaluate the density at specific points for test data
    x_grid_test = np.linspace(min(Y_test_array.flatten()), max(Y_test_array.flatten()), 100)
    density_values_test = kde_test.evaluate(x_grid_test)

    # Generate a PDF for visualization
    x_values_test = np.linspace(min(Y_test_array.flatten()), max(Y_test_array.flatten()), 100)
    pdf_values_test = kde.evaluate(x_values)

    num_samples = len(Y_test)  # Adjusted to the length of Y_test
    individual_forecasts = kde.resample(size=num_samples).flatten()

    # Set the figure size and DPI
    plt.figure(figsize=(8, 4.5), dpi=300)
    plt.plot(x_values, pdf_values, color=color, label='Predictions KDE')
    plt.plot(x_values_test, pdf_values_test, color=dark_gray, label="True values")
    plt.xlim(-50, 150)
    plt.xlabel(xlabel)
    plt.ylabel('Estimated probability density')
    plt.legend()
    plt.show()

    # R2
    Y_train_pred = RForregCV.predict(X_train)
    Y_train_dev = sum((Y_train-Y_train_pred)**2)
    r2 = 1 - Y_train_dev/Y_train_meandev
    print("R2 =", r2)

    # Pseudo R2
    Y_test_pred = RForregCV.predict(X_test)
    Y_test_dev = sum((Y_test-Y_test_pred)**2)
    pseudor2 = 1 - Y_test_dev/Y_test_meandev
    print("Pseudo-R2 =", pseudor2)

    # Mean Absolute Error (MAE)
    mae = mean_absolute_error(Y_test, Y_test_pred)
    print(f"Mean Absolute Error (MAE): {mae}")

    # Mean Squared Error (MSE)
    mse = mean_squared_error(Y_test, Y_test_pred)
    print(f"Mean Squared Error (MSE): {mse}")

    # Calculate RMSE
    rmse = np.sqrt(mse)
    print(f"Root Mean Squared Error (RMSE): {rmse}")

    # Continuous Ranked Probability Score (CRPS)
    pred = individual_forecasts
    test = Y_test.values.flatten()
    y_pred_prob = calculate_cdf(combined_predictions)
    crps_old = np.mean(calculate_crps(y_pred_prob, Y_test))
    crps = ps.crps_ensemble(test, forecasts=pred)
    crps_mean = np.mean(crps)
    crps_median = np.median(crps)
    print(f"Median CRPS: {crps_median}")
    print(f"Continuous Ranked Probability Score (CRPS): {crps_mean}")
    print(f"CRPS OLD: {crps_old}")

    report.loc[len(report)] = ['Random ForestCV',
                               r2,
                               pseudor2,
                               CV_rfmodel.cv_results_['mean_test_score'][CV_rfmodel.best_index_],
                               CV_rfmodel.cv_results_['std_test_score'][CV_rfmodel.best_index_],
                               mae,
                               mse,
                               rmse,
                               crps_mean,
                               data_set]

    xlabel = ''
    color = 'green'
    lightorange = '#ffcc66'

    if data_set_id % 2 == 0:
        xlabel = 'Arrival delay - STD at 0 [min]'
        color = 'lightblue'
    else:
        xlabel = 'Departure delay - STA at 0 [min]'
        color = lightorange

    # Calculate the number of bins with 1-minute width
    num_bins = int((150 - (-50)) / 1)

    # Set the figure size and DPI
    plt.figure(figsize=(8, 4.5), dpi=300)

    # Initialize an array to accumulate histograms for predictions
    combined_histogram = np.zeros(num_bins - 1)

    # Convert individual tree predictions to histograms and accumulate
    for i, predictions in enumerate(tree_predictions):
        histogram, _ = np.histogram(predictions, bins=np.linspace(-50, 150, num=num_bins), density=True)
        combined_histogram += histogram

    # Normalize the combined histogram so that the probabilities sum to 1.0
    combined_histogram /= np.sum(combined_histogram)

    # Plot the combined histogram for predictions
    plt.bar(
        np.linspace(-50, 150, num=num_bins-1),
        combined_histogram,
        width=1,
        color=color,
        edgecolor='white',
        linewidth=0.5,
        label='Predictions'
    )

    # Create a histogram for true values (Y_test)
    true_histogram, _ = np.histogram(Y_test, bins=np.linspace(-50, 150, num=num_bins), density=True)

    # Plot the histogram for true values
    plt.bar(
        np.linspace(-50, 150, num=num_bins-1),
        true_histogram,
        width=1,
        color=dark_gray,
        alpha=0.5,
        edgecolor='white',
        linewidth=0.5,
        label='True values'
    )

    plt.xlabel(xlabel)
    plt.xlim(-50, 150)
    plt.xticks(np.arange(-50, 151, 25))
    plt.ylabel('Probability')
    plt.legend()  # Add legend to distinguish between predictions and true values
    plt.show()

    # Set the figure size and DPI
    plt.figure(figsize=(8, 4.5), dpi=300)

    # Convert individual tree predictions to histograms
    for i, predictions in enumerate(tree_predictions):
        plt.hist(predictions, bins=np.linspace(-50, 150, num=num_bins), density=True, alpha=0.5)

    plt.xlabel(xlabel)
    plt.xlim(-50, 150)
    plt.xticks(np.arange(-50, 151, 25))
    plt.ylabel('Probability')
    plt.show()

    # Set the figure size and DPI
    fig, ax = plt.subplots(figsize=(8, 4.5), dpi=300)
    delay_range = np.arange(-50, 151, 1)

    # Plot the combined histogram with equal-width bins and a white border
    ax.hist(combined_predictions, bins=delay_range, density=True, label='Combined predictions', color=color, edgecolor='white', align='mid', linewidth=0.5)
    ax.hist(Y_test, bins=delay_range, density=True, alpha=0.5, label='True values', color=dark_gray, edgecolor='white', align='mid', linewidth=0.5)

    # Adding labels and title
    ax.set_xlabel(xlabel)
    ax.set_ylabel('Probability')
    ax.set_xlim(-50, 150)

    # Adding legend
    ax.legend(['Combined predictions', 'True values'], loc='upper right')
    plt.show()

    if data_set_id > 3:
        # Create a Shap explainer
        explainer = shap.Explainer(RForregCV)
        shap_values = explainer.shap_values(X_train)

        # Display a summary plot with a specified DPI
        dpi_value = 300  # Set your desired DPI value
        shap.summary_plot(shap_values, X_train, show=False)
        plt.gcf().set_dpi(dpi_value)
        plt.show()

    # Sample four data points from X_test as a pandas DataFrame
    test_data_points = [X_test.sample(n=1, random_state=i) for i in range(4)]

    for i, data_point in enumerate(test_data_points):
        # Display the selected random data point id
        id = data_point.index.item()
        print("Selected Random Data Point ID:")
        print(id)

        # Predictions
        flight_prediction = RForregCV.predict(data_point)
        actual_value = Y_test.loc[id]  # Assuming Y_test is a Series

        # Get predictions from individual trees
        data_tree_predictions = np.stack([tree.predict(data_point) for tree in RForregCV.estimators_])

        # Combine predictions (mean for regression)
        combined_predictions = np.mean(data_tree_predictions, axis=0)

        # Set the figure size and DPI
        plt.figure(figsize=(4, 3), dpi=300)

        # Plot a histogram of the combined predictions
        hist, bin_edges, _ = plt.hist(
            data_tree_predictions,
            bins=np.linspace(-50, 150, num=num_bins),  # Specify bin edges
            color=colors[i][0],
            label='Predictions',
            density=True,
            alpha=0.7
        )

        # Scatter plot for the regression prediction
        scatter = plt.scatter(flight_prediction[0], 0, marker='^', color=colors[i][1], s=250, label='Prediction value')

        # Line for the actual value
        line = plt.axvline(x=actual_value, color=colors[i][2], linestyle='dashed', linewidth=2, label='Actual value')

        # Set x-axis limits
        plt.xlim(-50, 150)
        plt.xlabel(xlabel)
        plt.ylim(0, 0.6)
        plt.ylabel('Probability')

        # Set x-axis ticks to 5-minute intervals
        plt.xticks(np.arange(-50, 151, 50))

        # Set y-axis ticks to 0.05 intervals
        plt.yticks(np.arange(0, 0.61, 0.15))
        plt.title(f'Flight {id}')

        # Create a custom legend with handles and labels
        legend_handles = [plt.Rectangle((0,0),1,1, color=colors[i][0]), scatter, line]
        legend_labels = ['RF Predictions', 'RF Prediction value', 'Actual value']
        plt.legend(legend_handles, legend_labels, loc='upper right', fontsize=10, markerscale=0.5)
        plt.show()

## Mixture Density Network

In [ ]:
def sample_from_mdn(params, num_samples=100):
    mus, sigmas, pis = np.split(params, 3, axis=-1)

    # Reshape pis to match the shape of mus and sigmas
    pis = np.reshape(pis, mus.shape)

    # Ensure probabilities are non-negative and sum to 1
    pis = np.maximum(pis, 0)
    pis /= np.sum(pis, axis=-1, keepdims=True)

    # Choose components based on the mixture probabilities
    chosen_components = np.random.choice(mus.shape[-1], size=num_samples, p=pis[0])

    # Sample from the chosen components
    samples = mus[0, chosen_components] + sigmas[0, chosen_components] * np.random.randn(num_samples)

    return samples

def mdn_loss(y_true, params):
    mu, sigma, alpha = tf.split(params, 3, axis=1)

    # Clip probabilities to avoid numerical instability
    alpha = tf.clip_by_value(alpha, clip_value_min=1e-5, clip_value_max=0.999)

    # Clip standard deviations (sigma)
    sigma = tf.clip_by_value(sigma, clip_value_min=1e-5, clip_value_max=10.0)

    gm = tfp.distributions.MixtureSameFamily(
        mixture_distribution=tfp.distributions.Categorical(probs=alpha),
        components_distribution=tfp.distributions.Normal(loc=mu, scale=sigma)
    )
    return -tf.reduce_mean(gm.log_prob(y_true))

class MDN(tf.keras.Model):
    def __init__(self, num_components=30):
        super(MDN, self).__init__()
        self.num_components = num_components
        self.dense1 = layers.Dense(256, activation='relu')
        self.dense2 = layers.Dense(256, activation='relu')
        self.dense3 = layers.Dense(128, activation='relu')
        self.dense4 = layers.Dense(128, activation='relu')
        self.mixture_layer = layers.Dense(self.num_components * 3)

    def call(self, inputs):
        x = self.dense1(inputs)
        x = self.dense2(x)
        x = self.dense3(x)
        x = self.dense4(x)
        params = self.mixture_layer(x)
        return params

for X_train, Y_train, X_test, Y_test, Y_train_meandev, Y_test_meandev, data_set, data_set_id in datasets:
    print("-------- Data Set:", data_set, "--------")
    # Create an instance of the MDN model
    mdn_model = MDN(num_components=30)

    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    mdn_model.compile(optimizer=optimizer, loss=mdn_loss)

    # Train the MDN model
    mdn_model.fit(X_train, Y_train)

    # Make predictions using the trained MDN model
    params = mdn_model.predict(X_test)
    mu, sigma, alpha = tf.split(params, 3, axis=1)

    # Sample from the MDN to get predictions
    categorical_distribution = tfp.distributions.Categorical(probs=alpha)
    samples = tf.convert_to_tensor(categorical_distribution.sample(len(X_test)))
    mdn_predictions = tf.gather(mu, samples)

    # Collapse the num_components dimension by taking the mean
    mdn_predictions_mean = tf.reduce_mean(mdn_predictions, axis=1)
    # Select the first column of mdn_predictions
    mdn_predictions_col = mdn_predictions_mean[:, 0]

    # Calculate R2
    r2 = r2_score(Y_test, mdn_predictions_col)
    print("R2 =", r2)

    # Pseudo R2
    y_test_mean = np.mean(Y_test)
    pseudo_r2 = 1 - (np.sum((mdn_predictions_col - Y_test)**2) / np.sum((y_test_mean - Y_test)**2))
    print("Pseudo-R2 =", pseudo_r2)

    # Mean Absolute Error (MAE)
    mae = mean_absolute_error(Y_test, mdn_predictions_col)
    print(f"Mean Absolute Error (MAE): {mae}")

    # Mean Squared Error (MSE)
    mse = mean_squared_error(Y_test, mdn_predictions_col)
    print(f"Mean Squared Error (MSE): {mse}")

    # Calculate RMSE
    rmse = np.sqrt(mse)
    print(f"Root Mean Squared Error (RMSE): {rmse}")

    # Continuous Ranked Probability Score (CRPS)
    cdf_values = norm.cdf(X_test, loc=mu, scale=sigma)
    crps = np.mean(ps.crps_ensemble(Y_test, mdn_predictions_col))
    print(f"Continuous Ranked Probability Score (CRPS): {crps}")

    xlabel = ''
    color = 'green'
    lightorange = '#ffcc66'

    if data_set_id % 2 == 0:
        xlabel = 'Arrival delay - STD at 0 [min]'
        color = 'lightblue'
    else:
        xlabel = 'Departure delay - STA at 0 [min]'
        color = lightorange

    # Assuming 'params' is your MDN output
    samples = sample_from_mdn(params, num_samples=1000)

    # Plot the KDE of the samples
    plt.figure(figsize=(8, 4.5), dpi=300)
    sns.kdeplot(samples, color=color, label='Predicted PDF', fill=True)
    sns.kdeplot(Y_test, color=dark_gray, alpha=0.5, label='True values', fill=True)
    plt.xlim(-50, 150)
    plt.xlabel(xlabel)
    plt.ylabel('Probability density')
    # plt.title('Probability Density Function (PDF) of MDN')
    plt.legend()
    plt.show()

    # Set up plot
    plt.figure(figsize=(8, 4.5), dpi=300)

    # Plot KDE for the entire set of predictions
    sns.kdeplot(mu[0], color=color, label='Predictions')
    # Plot KDE for the actual values (Y_test)
    sns.kdeplot(Y_test, color=dark_gray, alpha=0.5, label='True values')
    plt.xlim(-50, 150)
    plt.xlabel(xlabel)
    plt.ylim(0, 1)
    plt.ylabel('Probability density')
    plt.legend()
    plt.show()

    # Wrap the MDN model's predict function with a function that outputs only mu
    def predict_mu(X):
        params = mdn_model.predict(X)
        mu, _, _ = tf.split(params, 3, axis=1)
        return mu.numpy()

    if data_set_id > 3:
        # Use shap.kmeans to summarize the background data
        background_summary = shap.kmeans(X_train, 10)  # You can adjust the number of means (K) as needed

        # Use the KernelExplainer with the summarized background data
        explainer = shap.KernelExplainer(predict_mu, background_summary)
        shap_values = explainer.shap_values(X_test[:100], nsamples=100)  # Use a subset of samples for visualization

        # Set the figure size with DPI and plot the SHAP values for the first 100 samples
        plt.figure(figsize=(10, 6), dpi=300)  # Adjust the size and DPI as needed
        shap.summary_plot(shap_values, X_test[:100], show=False, plot_size=[10,6])
        # Move the legend outside the plot area
        plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

        # Display the plot
        plt.show()

    # Sample one data point from X_test as a pandas DataFrame
    test_data_points = [X_test.sample(n=1, random_state=i) for i in range(4)]

    for i, data_point in enumerate(test_data_points):
        # Display the selected random data point id
        id = data_point.index.item()
        print("Selected Random Data Point ID:")
        print(id)

        actual_value = Y_test.loc[id]  # Assuming Y_test is a Series

        # Extract the features from the data point
        x_data_point = data_point.values

        # Make predictions using the trained MDN model
        params_i = mdn_model.predict(x_data_point)
        mu_i, sigma_i, alpha_i = tf.split(params_i, 3, axis=1)
        samples_i = sample_from_mdn(params_i, num_samples=1000)

        # Set up plot
        plt.figure(figsize=(4, 3), dpi=300)

        # Plot the PDF for the current data point
        kde_plot = sns.kdeplot(samples_i, color=colors[i][0], label="MDN Predictions KDE", fill=True)

        # Scatter plot for the mean of the PDF
        mean_value = tf.reduce_mean(samples_i)
        scatter = plt.scatter(mean_value, 0, marker='^', color=colors[i][1], s=250, label='Mean value of PDF')

        # Line for the actual value
        line = plt.axvline(x=actual_value, color=colors[i][2], linestyle='dashed', linewidth=2, label='Actual value')

        # Set x-axis limits
        plt.xlim(-50, 150)
        plt.xlabel(xlabel)
        plt.ylim(0, 0.6)
        plt.ylabel('Probability density')
        # Set x-axis ticks to 5-minute intervals
        plt.xticks(np.arange(-50, 151, 50))

        # Set y-axis ticks to 0.05 intervals
        plt.yticks(np.arange(0, 0.61, 0.15))
        plt.title(f'Flight {id}')

        # Create a custom legend with handles and labels
        legend_handles = [
            Line2D([0], [0], color=colors[i][0], linewidth=2),
            scatter,
            line
        ]
        legend_labels = ['MDN Predictions', 'MDN Prediction value', 'Actual value']
        plt.legend(legend_handles, legend_labels, loc='upper right', fontsize=10, markerscale=0.5)
        plt.show()

## Final Report

Get the indices of the random flights

In [ ]:
for X_train, Y_train, X_test, Y_test, Y_train_meandev, Y_test_meandev, data_set, data_set_id in datasets:
    # print("-------- Data Set:", data_set, "--------")
    # Sample one data point from X_test as a pandas DataFrame
    test_data_points = [X_test.sample(n=1, random_state=i) for i in range(4)]
    indices = []

    for i, data_point in enumerate(test_data_points):
        # Display the selected random data point id
        id = data_point.index.item()
        indices.append(id)

    print(indices)

Print the report

In [ ]:
print(report)